In [12]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import os, pickle
from tqdm import tqdm
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict, Counter

import logging
logger = logging.getLogger(__name__)
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [2]:
DATA_PATH = 'data_cache/'

In [89]:
def create_embedding_matrix(sparse_columns, varlen_sparse_columns, embed_dim,
                            init_std=0.0001, padding=True, device='cpu', mode='mean'):
    # sparse_columns => dict{'name':vocab_size}
    # Return nn.ModuleDict: for sparse features, {embedding_name: nn.Embedding}
    padding_idx = 0 if padding else None
    sparse_embedding_dict = {
        feat: nn.Embedding(sparse_columns[feat], embed_dim, padding_idx=padding_idx)
                             for feat in sparse_columns
    }
    
    if varlen_sparse_columns:
        varlen_sparse_embedding_dict = {
            feat:nn.EmbeddingBag(varlen_sparse_columns[feat], embed_dim, padding_idx=padding_idx,
                                 mode=mode) for feat in varlen_sparse_columns
        }
        sparse_embedding_dict.update(varlen_sparse_embedding_dict)
        
    embedding_dict = nn.ModuleDict(sparse_embedding_dict)
    
    for tensor in embedding_dict.values():
        nn.init.normal_(tensor.weight, mean=0, std=init_std)
        # nn.init.kaiming_uniform_(tensor.weight, mode='fan_in', nonlinearity='relu')

    return embedding_dict.to(device)


class EGES(nn.Module):
    def __init__(self, sparse_dict, varlen_sparse_dict=None, target_col='sku_id',
                 n_embed=64, k_side=3, noise_dist=None, device='cpu', padding=True):
        """sparse_dict: dict, {feature_name: vocab_size}
        """
        super().__init__()
        self.n_embed = n_embed
        self.k_side = k_side
        self.device = device
        self.padding = padding
        self.target_col = target_col
        self.features = list(sparse_dict.keys())
        if varlen_sparse_dict:
            self.features = self.features + list(varlen_sparse_dict.keys())
        # 如果padding了的话，则负采样出来的index均需要+1
        self.sample_word_offset = 1 if padding else 0
        # input embedding dict, include item and side info
        self.input_embedding_dict = create_embedding_matrix(
            sparse_dict, varlen_sparse_dict, n_embed,
            init_std=0.0001, padding=padding, device=device, mode='mean')
        self.out_embed = nn.Embedding(sparse_dict[target_col], n_embed,
                                      padding_idx=0 if padding else None)
        self.attn_embed = nn.Embedding(sparse_dict[target_col], k_side+1, 
                                       padding_idx=0 if padding else None)
        
        # Initialize out embedding tables with uniform distribution
        nn.init.normal_(self.out_embed.weight, mean=0, std=0.0001)
        nn.init.normal_(self.attn_embed.weight, mean=0, std=0.0001)

        if noise_dist is None:
            # sampling words uniformly
            self.noise_dist = torch.ones(self.n_vocab)
        else:
            self.noise_dist = noise_dist
        self.noise_dist = self.noise_dist.to(device)

    def forward_input(self, input_dict):
        # return input vector embeddings
        embed_lst = []
        for col in self.features:
            if col in input_dict:
                input_vector = self.input_embedding_dict[col](input_dict[col])
                embed_lst.append(input_vector)

        batch_size = input_vector.shape[0]
        # embeds => [batch_size, k_side+1, n_embed]
        embeds = torch.cat(embed_lst, dim=1).reshape(batch_size, self.k_side+1, self.n_embed)
        
        # attation => [batch_size, k_side+1]
        attn_w = self.attn_embed(input_dict[self.target_col])
        attn_w = torch.exp(attn_w)
        attn_s = torch.sum(attn_w, dim=1).reshape(-1, 1)
        attn_w = (attn_w/attn_s).reshape(batch_size, 1, self.k_side+1) # 归一化
        
        # attw => [batch_size, 1, k_side+1]
        # embeds => [batch_size, k_side+1, embed_size]
        # matmul out => [batch_size, 1, embed_size]
        input_vector = torch.matmul(attn_w, embeds).squeeze(1)
        
        return input_vector

    def forward_output(self, output_words):
        # return output vector embeddings 
        output_vector = self.out_embed(output_words)
        return output_vector
    
    def forward_noise(self, batch_size, n_samples):
        """Generate noise vectors with shape [batch_size, n_samples, n_embed]
        """
        # sample words from our noise distribution 
        noise_words = torch.multinomial(self.noise_dist, batch_size*n_samples, 
                                        replacement=True) + self.sample_word_offset
        noise_vector = self.out_embed(noise_words).view(batch_size, n_samples, self.n_embed)
        
        return noise_vector
    
    def forward_cold(self, input_dict):
        """处理冷启动item，使用其side info Embedding的均值
        """
        # return input vector embeddings
        embed_lst = []
        for col in self.features:
            if col in input_dict:
                input_vector = self.input_embedding_dict[col](input_dict[col])
                embed_lst.append(input_vector)

        batch_size = input_vector.shape[0]
        # embeds => [batch_size, k_side, n_embed]
        embeds = torch.cat(embed_lst, dim=1).reshape(batch_size, self.k_side, self.n_embed)
        return torch.mean(embeds, dim=1)


class NegativeSamplingLoss(nn.Module):
    """这里用的是负对数似然, 而不是sampled softmax
    """
    def __init__(self):
        super().__init__()
    
    def forward(self, input_vectors, output_vectors, noise_vectors):
        batch_size, embed_size = input_vectors.shape
        
        # input vectors should be a batch of column vectors
        input_vectors = input_vectors.view(batch_size, embed_size, 1)
        
        # output vectors should be a batch of row vectors
        output_vectors = output_vectors.view(batch_size, 1, embed_size)
        
        # bmm = batch matrix multiplication
        # target words log-sigmoid loss
        out_loss = torch.bmm(output_vectors, input_vectors).sigmoid().log()
        
        # negative sampling words log-sigmoid loss
        # negative words sigmoid optmize to small, thus here noise_vectors.neg()
        noise_loss = torch.bmm(noise_vectors.neg(), input_vectors).sigmoid().log()
        # sum the losses over the sample of noise vectors
        noise_loss = noise_loss.squeeze().sum(1)
        
        # sum target and negative loss
        return -(out_loss + noise_loss).mean()


class TextData(Dataset):
    def __init__(self, df, sparse_columns=['feedid','label','authorid','feed_machine_tag_tfidf_cls_32',
                                           'feed_machine_kw_tfidf_cls_17'],
                 varlen_sparse_columns=[], device='cpu'):
        self.sparse_columns = sparse_columns
        self.varlen_sparse_columns = varlen_sparse_columns
        self.device = device
        self.data = {
            col:df[col].values for col in sparse_columns
        }
        if varlen_sparse_columns:
            for col in varlen_sparse_columns:
                self.data[col] = np.vstack(df[col].values)

        self.data_num = len(df)
    
    def __len__(self):
        return self.data_num
    
    def __getitem__(self, idx):
        data_dic = {}
        for col in self.sparse_columns:
            data_dic[col] = torch.tensor(self.data[col][idx]).long() #.to(self.device)
        if self.varlen_sparse_columns:
            for col in self.varlen_sparse_columns:
                data_dic[col] = torch.tensor(self.data[col][idx, :]).long() #.to(self.device)

        return data_dic

## 准备数据

In [8]:
df_pair = pd.read_pickle(f'{DATA_PATH}/pairs.pkl')
df_pair.head(2)

,sku_id,label,brand,shop_id,cate
0,21864,11800,2347,4110,6
1,21864,28044,2347,4110,6


In [7]:
# 计算负采样时候各个word的概率
word_counts = pickle.load(open(f'{DATA_PATH}/word_counts.pkl', 'rb'))
# 按label encoder 进行排序，因为需要跟后面Embedding table采样保持一致
word_counts = sorted(word_counts, key=lambda x:x[0])
counts = np.array([wc[1] for wc in word_counts])

noise_dist = torch.from_numpy(counts**(0.75)/np.sum(counts**(0.75)))

In [10]:
# 各个field的维度，包含padding index
lbe_dict = pickle.load(open(f'{DATA_PATH}/label_dict.pkl', 'rb'))
vocab_dict = {feat:len(lbe_dict[feat].classes_)+1 for feat in lbe_dict}
vocab_dict

{'sku_id': 33345, 'brand': 3663, 'shop_id': 4786, 'cate': 80}

## 训练模型

In [25]:
device = 'gpu'
if device=='gpu' and torch.cuda.is_available():
    # print('cuda ready...')
    device = 'cuda:0'
else:
    device = 'cpu'

textdata = TextData(df_pair, sparse_columns=['sku_id','label','brand','shop_id','cate']) 
textloader = DataLoader(textdata,
                        batch_size=10000,
                        shuffle=True,
                        num_workers=10,
                        drop_last=False,
                        pin_memory=True)

embedding_dim = 128
model = EGES(vocab_dict, n_embed=embedding_dim, k_side=3, target_col='sku_id',
             noise_dist=noise_dist, device=device, padding=True).to(device)
criterion = NegativeSamplingLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# optimizer = torch.optim.Adagrad(model.parameters(), lr=0.01)

epoch = 10
for e in range(epoch):
    for i, data_dic in enumerate(textloader):
        # input, output and noise vectors
        data_dic = {feat:data_dic[feat].to(device) for feat in data_dic}
        input_vectors = model.forward_input(data_dic)
        output_vectors = model.forward_output(data_dic['label'])
        noise_vectors = model.forward_noise(data_dic['label'].shape[0], 10)
        # negative sampling loss
        loss = criterion(input_vectors, output_vectors, noise_vectors)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    logger.info(f'Epoch {e+1}/{epoch} Step {i} Loss = {loss}')

08/11/2021 15:18:51 - INFO - __main__ -   Epoch 1/10 Step 70 Loss = 2.9070849418640137
08/11/2021 15:18:57 - INFO - __main__ -   Epoch 2/10 Step 70 Loss = 1.984291434288025
08/11/2021 15:19:05 - INFO - __main__ -   Epoch 3/10 Step 70 Loss = 1.4065927267074585
08/11/2021 15:19:11 - INFO - __main__ -   Epoch 4/10 Step 70 Loss = 1.060995101928711
08/11/2021 15:19:18 - INFO - __main__ -   Epoch 5/10 Step 70 Loss = 0.8512248992919922
08/11/2021 15:19:24 - INFO - __main__ -   Epoch 6/10 Step 70 Loss = 0.6837941408157349
08/11/2021 15:19:31 - INFO - __main__ -   Epoch 7/10 Step 70 Loss = 0.6034356951713562
08/11/2021 15:19:37 - INFO - __main__ -   Epoch 8/10 Step 70 Loss = 0.5155255198478699
08/11/2021 15:19:44 - INFO - __main__ -   Epoch 9/10 Step 70 Loss = 0.46551644802093506
08/11/2021 15:19:51 - INFO - __main__ -   Epoch 10/10 Step 70 Loss = 0.4440944194793701


In [26]:
# 保存模型
torch.save(model.state_dict(), f'{DATA_PATH}/model.bin')

### 推理得到各个item的Embedding

In [132]:
df_item = df_pair.drop_duplicates(subset=['sku_id']).reset_index(drop=True)
print(df_item.shape)
df_item.head(2)

(32596, 5)


,sku_id,label,brand,shop_id,cate
0,21864,11800,2347,4110,6
1,11800,21864,748,1624,6


In [98]:
textdata1 = textdata = TextData(df_item, sparse_columns=['sku_id','label','brand','shop_id','cate']) 
textloader1 = DataLoader(textdata1,
                        batch_size=10000,
                        shuffle=False,
                        num_workers=10,
                        drop_last=False,
                        pin_memory=True)

embedding_dim = 128
model = EGES(vocab_dict, n_embed=embedding_dim, k_side=3, target_col='sku_id',
             noise_dist=noise_dist, device=device, padding=True).to(device)
state_dic = torch.load(f'{DATA_PATH}/model.bin')
model.load_state_dict(state_dic)
model = model.eval()

epoch = 10
emb_vectors = []
with torch.no_grad():
    for i, data_dic in enumerate(textloader1):
        # input, output and noise vectors
        data_dic = {feat:data_dic[feat].to(device) for feat in data_dic}
        input_vectors = model.forward_input(data_dic)
        emb_vectors.append(input_vectors.detach().cpu().numpy())

In [99]:
vocab_emb = dict(zip(df_item['sku_id'].tolist(), np.vstack(emb_vectors)))

In [101]:
len(vocab_emb)


32596

### 推理冷启动item的Embedding

In [102]:
df_item2 = df_pair.drop_duplicates(subset=['sku_id'])[['brand','shop_id','cate']].head(10)
textdata2 = textdata = TextData(df_item2, sparse_columns=['brand','shop_id','cate']) 
textloader2 = DataLoader(textdata2,
                        batch_size=10000,
                        shuffle=False,
                        num_workers=10,
                        drop_last=False,
                        pin_memory=True)

embedding_dim = 128
model = EGES(vocab_dict, n_embed=embedding_dim, k_side=3, target_col='sku_id',
             noise_dist=noise_dist, device=device, padding=True).to(device)
state_dic = torch.load(f'{DATA_PATH}/model.bin')
model.load_state_dict(state_dic)
model = model.eval()

epoch = 10
cold_vectors = []
with torch.no_grad():
    for i, data_dic in enumerate(textloader2):
        # input, output and noise vectors
        data_dic = {feat:data_dic[feat].to(device) for feat in data_dic}
        input_vectors = model.forward_cold(data_dic)
        cold_vectors.append(input_vectors.detach().cpu().numpy())

In [103]:
cold_emb = dict(zip(df_pair.drop_duplicates(subset=['sku_id'])['sku_id'].head(10).tolist(), np.vstack(cold_vectors)))
len(cold_emb)

10

In [104]:
cold_emb.keys()

dict_keys([21864, 11800, 28044, 4215, 31898, 8519, 21941, 14684, 2970, 11007])

### 查看冷启动方式得到的Embedding与原Embedding的相似度

In [111]:
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity

In [155]:
df_item.head(2)

,sku_id,label,brand,shop_id,cate
0,21864,11800,2347,4110,6
1,11800,21864,748,1624,6


In [152]:
sim1 = cosine_similarity(cold_emb[21864].reshape(1,-1), np.vstack(emb_vectors))
sim11 = list(zip(df_item['sku_id'].tolist(), sim1[0]))
sim11 = sorted(sim11, key=lambda x:x[1], reverse=True)
sim11[:5]

[(23870, 0.9760314),
 (1909, 0.9652012),
 (19325, 0.9550275),
 (4071, 0.95498335),
 (10087, 0.9508642)]

In [153]:
df_item.query('sku_id==23870')

,sku_id,label,brand,shop_id,cate
28829,23870,24721,2347,4110,6


In [158]:
df_item.query('(brand==2347)&(shop_id==4110)&(cate==6)').head()

,sku_id,label,brand,shop_id,cate
0,21864,11800,2347,4110,6
108,12520,2793,2347,4110,6
109,2793,12520,2347,4110,6
530,18844,19808,2347,4110,6
531,3126,19808,2347,4110,6


In [134]:
df_item.head(2)

,sku_id,label,brand,shop_id,cate
0,21864,11800,2347,4110,6
1,11800,21864,748,1624,6


In [149]:
sim2 = cosine_similarity(vocab_emb[21864].reshape(1,-1), np.vstack(emb_vectors))
sim22 = list(zip(df_item['sku_id'].tolist(), sim2[0]))
sim22 = sorted(sim22, key=lambda x:x[1], reverse=True)
sim22[:5]

[(21864, 1.0),
 (22840, 0.8361774),
 (12176, 0.8087917),
 (3813, 0.80076164),
 (13764, 0.764928)]

In [151]:
df_item.query('sku_id==22840')

,sku_id,label,brand,shop_id,cate
23867,22840,21864,3101,3000,6


In [160]:
# 原Embedding与cold Embedding最相似的word的相似度
# 相似度排名大概在100多（总32596）
cosine_similarity(vocab_emb[21864].reshape(1,-1), vocab_emb[23870].reshape(1,-1))

array([[0.59903204]], dtype=float32)

In [164]:
# cold Embedding与原Embedding 最相似的word的相似度
cosine_similarity(cold_emb[21864].reshape(1,-1), vocab_emb[22840].reshape(1,-1))

array([[0.7097324]], dtype=float32)

In [167]:
cosine_similarity(vocab_emb[23870].reshape(1,-1), vocab_emb[22840].reshape(1,-1))

array([[0.6926477]], dtype=float32)